In [4]:
import os
import re
import pymupdf
import django_setup
from IPython.display import Markdown, display
from bs4 import BeautifulSoup as bs

from django.conf import settings

from cmj.utils import clean_text

gemini_api_key = settings.GEMINI_API_KEY

In [5]:
import google.generativeai as genai
genai.configure(api_key=gemini_api_key)


In [6]:
def get_model_configured():
    generation_config = {
      "temperature": 0.1,
      "top_p": 0.95,
      "top_k": 40,
      #"max_output_tokens": 8192,
      "response_mime_type": "text/plain",
    }

    model = genai.GenerativeModel(
      model_name="gemini-2.0-flash-exp",
      #model_name="gemini-2.5-pro-exp-03-25",
      #model_name="gemini-2.5-flash-preview-04-17",
      generation_config=generation_config,
    )

    return model

model = get_model_configured()


In [7]:
def make_prompt(num_prompt, original, analisado, o_epigrafe, a_epigrafe):

    prompt0 = f"""
Assuma a personalidade de um especialista em produção de textos legislativos em uma câmara municipal,
com experiência em redação de documentos oficiais. Sua tarefa é analisar dois textos
que são pedidos de providências feitos por vereadores e identificar se eles estão pedindo o mesmo benefício.
Os textos podem ter diferenças de redação, mas você deve se concentrar no conteúdo e na intenção dos pedidos.
Para tal tarefa, compare o conteúdo de <ORIGINAL></ORIGINAL> com o conteúdo de <ANALISADO></ANALISADO>.
Para citar estes dois conteúdos, nomeie eles respectivamente da seguinte maneira:
"{o_epigrafe}" e "{a_epigrafe}".
Remova de sua análise os autores pois são irrelevantes para a comparação requerida.
O importante é o que está sendo pedido, quem será o beneficiário do pedido e para qual localidade
está sendo feito tal pedido.
Não faça considerações adicionais e ou mesmo conclusão extra. Neste contexto responda:

- Os textos estão pedindo o mesmo benefício para a mesma localidade?
- Descreva de forma sucinta e direta o que está sendo pedido em <ORIGINAL></ORIGINAL> e <ANALISADO></ANALISADO> informando também os beneficiários e as localidades citadas.
- Calcule a semelhança percentual entre os documentos desconsiderando autores, focando na solicitação e no beneficiário, qual semelhança percentual entre <ORIGINAL></ORIGINAL> e <ANALISADO></ANALISADO>? Coloque o resultado em percentual com uma marcação fácil de ser extraída via regex "\[\[ \d{1,3}% \]\]".
- formate a resposta em MARKDOWN, utilizando linguagem dissertativa com os títulos e subtítulos necessários para facilitar a leitura, utilizando negrito e itálico quando necessário
- Não utilize a palavra "plágio" em sua resposta, se necessário expressar tal sentido, utilize a palavra "similaridade".

<ORIGINAL>{original}</ORIGINAL>

<ANALISADO>{analisado}</ANALISADO>
    """

    prompt1 = f"""    Compare o conteúdo de <ORIGINAL></ORIGINAL>, ao qual será chamado daqui em diante de "T.O.", com o conteúdo de <ANALISADO></ANALISADO>, ao qual será chamado daqui em diante de "T.A.", e escreva em formato MARKDOWN respondendo as perguntas numeradas que estão em <PERGUNTAS></PERGUNTAS>.

<PERGUNTAS>
    1) "T.O." e "T.A." fazem solicitações a seus destinatários? Quais são essas solicitações? E qual o beneficiário de cada uma das solicitações?
    2) Desconsiderando autores e destinatários, focando na solicitação e no beneficiário, qual grau de semelhança, de zero a um, poderia ser atribuído entre "T.O." e "T.A."?
</PERGUNTAS>

<ORIGINAL>{original}</ORIGINAL>

<ANALISADO>{analisado}</ANALISADO>
    """

    prompt2 = f"""    Assuma a personalidade de um especialista em plágio de documentos. É muito importante que em sua resposta nunca seja usada a palavra "plágio", em vez disso, se necessário, utilize a palavra similaridade.
    Compare o conteúdo de <ORIGINAL></ORIGINAL>, ao qual será chamado daqui em diante de "T.O.", com o conteúdo de <ANALISADO></ANALISADO>, ao qual será chamado daqui em diante de "T.A." e escreva em formato MARKDOWN respondendo as perguntas numeradas que estão em <PERGUNTAS></PERGUNTAS>. Seja Conciso.

<PERGUNTAS>
    1) Existe semelhança temática entre "T.O." e "T.A."? Quais são estas semelhanças? É o mesmo tema (frize)?
    2) Existe semelhança nas solicitações feita em "T.O." e "T.A."? Quais são estas semelhanças?
    3) A solicitações feitas é uma melhoria para algum lugar, seja esse lugar um logradouro público, um bairro, um prédio comercial, um prédio público, ou ainda instituições públicas e/ou privadas. Que lugar é este? É o mesmo lugar?
    4) Alguma consideração a fazer quanto a similaridade entre "T.O." e "T.A."?
    5) Desconsiderando autores e destinatários, focando na solicitação e no beneficiário, qual grau de semelhança, de zero a um, poderia ser atribuído entre "T.O." e "T.A."?
</PERGUNTAS>

<ORIGINAL>{original}</ORIGINAL>

<ANALISADO>{analisado}</ANALISADO>
    """

    prompts = [prompt0, prompt1, prompt2]

    prompt = prompts[num_prompt]
    while '  ' in prompt:
        prompt = prompt.replace('  ', ' ')

    return prompt


In [9]:
from sapl.materia.models import MateriaLegislativa

originais = MateriaLegislativa.objects.filter(numero=322, tipo_id=3, ano=2025).order_by('id')
analisadas = MateriaLegislativa.objects.filter(numero=252, tipo_id=3, ano=2025).order_by('id')

fez_um_doc = False
for original in originais:
    for analisada in analisadas:
        if original == analisada:
            continue

        if not set(original.autores.all()) - set(analisada.autores.all()):
            continue

        fez_um_doc = True

        print(original, '≃', analisada)

        doc_original = pymupdf.open(original.texto_original.original_path)
        text_original = ' '.join([page.get_text() for page in doc_original])
        text_original = clean_text(text_original)

        doc_analisada = pymupdf.open(analisada.texto_original.original_path)
        text_analisada = ' '.join([page.get_text() for page in doc_analisada])
        text_analisada = clean_text(text_analisada)

        prompt = make_prompt(0, text_original, text_analisada, original.epigrafe_short, analisada.epigrafe_short)

        answer = model.generate_content(prompt)
        display(Markdown(answer.text))

        if fez_um_doc:
            break

    if fez_um_doc:
        break


Requerimento nº 322 de 2025 ≃ Requerimento nº 252 de 2025


- Os textos não estão pedindo o mesmo benefício para a mesma localidade.

- **REQ 322/2025:** Solicita estudo de viabilidade para implantação do Programa Municipal de Atenção à Pessoa em Situação de Dependência Química e Alcoólica, nos moldes do Programa Mão Amiga. Os beneficiários são pessoas em situação de dependência química e alcoólica no município de Jataí-GO.
- **REQ 252/2025:** Solicita a realização de um mutirão para a entrega de escrituras habitacionais. Os beneficiários são famílias que aguardam a regularização documental de seus imóveis no município de Jataí-GO.

- Semelhança percentual entre os documentos: [[ 0% ]]


In [ ]:
from sapl.materia.models import MateriaLegislativa

requerimentos = MateriaLegislativa.objects.filter(
    tipo_id=3, ano=2025, similaridades__isnull=True
).prefetch_related('autores', 'assuntos'
                   ).order_by('-id').distinct()


In [ ]:
# cria uma lista de tuplas contendo o id do requerimento, uma tumpla com os ids dos autores e uma tupla com os ids dos assuntos
requerimentos_ids = []
for requerimento in requerimentos:
    requerimentos_ids.append(
        (requerimento.id, tuple(requerimento.autores.values_list('id', flat=True)),
         tuple(requerimento.assuntos.values_list('id', flat=True)))
    )

# ordena a lista pela quantidade de assuntos
requerimentos_ids = sorted(requerimentos_ids, key=lambda x: (len(x[2]), len(x[1])), reverse=True)

# ordena internamente cada tupla e assuntos
for i, r in enumerate(requerimentos_ids):
    requerimentos_ids[i] = (r[0], tuple(sorted(r[1])), tuple(sorted(r[2])))

# cria uma lista dois a dois de todos os requerimentos com todos menos com ele mesmo, e se o autor for o mesmo
requerimentos_comparacao = {}
for i, r1 in enumerate(requerimentos_ids):
    for j, r2 in enumerate(requerimentos_ids):
        if i == j:
            continue
        if r1[1] == r2[1]:
            continue
        if len(r1[1]) >=5 or len(r2[1]) >= 5:
            continue
        set1 = set(r1[2])
        set2 = set(r2[2])
        intersection = set1.intersection(set2)
        requerimentos_comparacao[(r1[0], r2[0])] = tuple(intersection)

# remove os requerimentos que possuem chave invertida
# ou seja, dado (a, b), remove (b, a)

requerimentos_comparados = {}
for k, v in requerimentos_comparacao.items():
    if (k[1], k[0]) in requerimentos_comparados:
        continue
    requerimentos_comparados[k] = v


# ordena a comparação pela quantidade de elementos na interseção
requerimentos_comparados = sorted(requerimentos_comparados.items(), key=lambda x: len(x[1]), reverse=True)
print(len(requerimentos_ids), len(requerimentos_comparados))

In [ ]:
for r in requerimentos_comparados[:100]:
    print(r)

In [ ]:
def get_requerimento_in_requerimentos_comparados(r):
    items = []
    for i, r1 in enumerate(requerimentos_comparados):
        if r in r1[0]:
            items.append(r1)
    return items

aparicoes = get_requerimento_in_requerimentos_comparados(21536)


In [ ]:
list(aparicoes)

In [2]:
from sapl.base.tasks import task_analise_similaridade_entre_materias_function
from sapl.materia.models import AnaliseSimilaridade

#task_analise_similaridade_entre_materias_function()

AnaliseSimilaridade.objects.filter(similaridade__gte=0).count()



505

In [16]:

analises = AnaliseSimilaridade.objects.filter(
    similaridade__gt=0
    ).order_by('-similaridade')

for analise in analises[:20]:
    print(analise.materia_1_id, analise.materia_2_id, analise.qtd_assuntos_comuns, analise.similaridade, analise, )
    #display(Markdown(analise.analise))
    #break

21861 21817 5 95 Matéria 1: Requerimento nº 300 de 2025 - Matéria 2: Requerimento nº 275 de 2025 - Similaridade: 95
21612 21555 5 75 Matéria 1: Requerimento nº 120 de 2025 - Matéria 2: Requerimento nº 69 de 2025 - Similaridade: 75
21537 21501 6 75 Matéria 1: Requerimento nº 62 de 2025 - Matéria 2: Requerimento nº 28 de 2025 - Similaridade: 75
21537 21623 5 70 Matéria 1: Requerimento nº 62 de 2025 - Matéria 2: Requerimento nº 129 de 2025 - Similaridade: 70
21826 21752 5 70 Matéria 1: Requerimento nº 279 de 2025 - Matéria 2: Requerimento nº 225 de 2025 - Similaridade: 70
21912 21555 5 70 Matéria 1: Requerimento nº 334 de 2025 - Matéria 2: Requerimento nº 69 de 2025 - Similaridade: 70
21914 21817 5 70 Matéria 1: Requerimento nº 336 de 2025 - Matéria 2: Requerimento nº 275 de 2025 - Similaridade: 70
21625 21817 5 60 Matéria 1: Requerimento nº 131 de 2025 - Matéria 2: Requerimento nº 275 de 2025 - Similaridade: 60
21625 21802 5 60 Matéria 1: Requerimento nº 131 de 2025 - Matéria 2: Requerim

In [18]:
import json
json.dumps({
            'materia_1_id': analise.materia_1_id,
            'materia_2_id': analise.materia_2_id,
            'analise': display(Markdown(analise.analise))
        })

### Análise Comparativa de Textos Legislativos

*Os textos estão pedindo o mesmo benefício para a mesma localidade?*

Não, os textos não estão pedindo o mesmo benefício para a mesma localidade.

*Descrição sucinta dos pedidos, beneficiários e localidades:*

*   **REQ 275/2025:** Solicita a construção de uma praça pública com equipamentos de lazer e esportes no Setor Residencial Jardim dos Ipês. Os beneficiários são os moradores do Setor Residencial Jardim dos Ipês, em Jataí, Goiás.
*   **REQ 261/2025:** Solicita a implantação de uma praça no Setor Recanto da Mata. Os beneficiários são os moradores do Setor Recanto da Mata, em Jataí, Goiás.

*Semelhança percentual entre os documentos:*

[[ 60% ]]


'{"materia_1_id": 21817, "materia_2_id": 21802, "analise": null}'

In [6]:
from django_celery_beat.models import PeriodicTask, IntervalSchedule, PeriodicTasks
p = PeriodicTask.objects.first()
p.queue = 'cq_base'
p.save()


In [7]:
p.__dict__

{'_state': <django.db.models.base.ModelState at 0x7949d17e7410>,
 'id': 1,
 'name': 'Analise de similaridade',
 'task': 'sapl.base.tasks.task_analise_similaridade_entre_materias',
 'interval_id': 1,
 'crontab_id': None,
 'solar_id': None,
 'clocked_id': None,
 'args': '[]',
 'kwargs': '{}',
 'queue': 'cq_base',
 'exchange': None,
 'routing_key': None,
 'headers': '{}',
 'priority': None,
 'expires': None,
 'expire_seconds': None,
 'one_off': False,
 'start_time': None,
 'enabled': True,
 'last_run_at': None,
 'total_run_count': 0,
 'date_changed': datetime.datetime(2025, 5, 19, 17, 4, 17, 855606, tzinfo=datetime.timezone.utc),
 'description': ''}

In [14]:
AnaliseSimilaridade.objects.filter(materia_1_id=21889,materia_2_id=21792)

<QuerySet []>